In [ ]:
#순서대로 읽어서 
#번호 순으로 저장

In [ ]:
! apt install tesseract-ocr
! apt install libtesseract-dev
! pip install Pillow
! pip install pytesseract
! sudo apt-get install tesseract-ocr-script-hang tesseract-ocr-script-hang-vert

In [ ]:
#라이브러리 임포트
import numpy as np
import cv2 #openCV package
from google.colab.patches import cv2_imshow
import re
import pytesseract
import glob
import os
import time
from PIL import ImageEnhance, ImageFilter, Image
#from pytesseract import Output
from matplotlib import pyplot as plt


#구글 드라이브와 연동 
from google.colab import drive
drive.mount('/gdrive')


In [ ]:
#수능완성 page trim 
def imtrim(page,page_num):

    #이미지 흑백화 

    page_num = int(page_num)

    #input으로 넣을 문제집 마다 다른 값 입력이 필요 
    #짝수 page 
    if (page_num%2 == 0): 
        left = page[150:-130, 100:855]
        right = page[150:-130,880:-142]


    #홀수 page 
    else:
        left = page[150:-130, 145:900]
        right = page[150:-130,905:-117]

    return left,right

#반페이지를 입력받고 크롭하기 
def contour(page_rl):


    imgray = cv2.cvtColor(page_rl, cv2.COLOR_BGR2GRAY) 


    #추출하려는 이미지 -> 기준점이 되는 이미지 경로입력 
    template = cv2.imread('insert your image path',0)
    w, h = template.shape[::-1]

    # 추출이미지를 찾아내기 
    res = cv2.matchTemplate(imgray,template,cv2.TM_CCOEFF_NORMED)
    threshold = 0.5
    loc = np.where( res >= threshold)


    #
    y_now = 0
    pt_now = 0

    print(loc)
    #
    for pt in zip(*loc[::-1]):

        pt_past = pt_now
        pt_now = pt[1]


        y_past = y_now

        y_now = pt_now + h + 20

        #
        if (pt_now >= pt_past+3): #a lot of +1 kind of same pts
            img_trim = page_rl[y_past:y_now,:]
            include(img_trim,qnum)

    #
    img_trim = page_rl[y_now:,:]
    include(img_trim,qnum)




def include(cropped,num):
    
    #
    global qnum
    number = str(num)
    #파일 이름 형식을 맞추기 위한 것 -> 사용시 이름을 다음형식으로 주지 않을 것이라면 사용하지 않아도 됨 
    if (len(number)==1):
        number = '000'+str(num)
    elif (len(number)==2):
        number = '00'+str(num)
    else:
        number = '0'+str(num)


    newfile=save_path+str(number)+"_pg"+j+".png"
    #print(newfile)
    #print(num)

    cv2.imwrite(newfile,cropped)
    #
    qnum=qnum+1
    

#문제집이 수능완성인 경우에 경로지정 
def make_path_SW():

    book_list=['2021','2020']

    #수능완성 
    for i in range(len(book_list)):
        path=book_list[i]+'/'
        path_list.append(path)
    

#문제집이 모의고사일 경우의 경로지정 
def make_path():
    year_list='2021','2020','2019','2018','2017']
    month_list=['06_','09_','sn_']
    type_list=['ga','na']
    book_list=[]
    for i in range(len(year_list)):
        for j in range(len(month_list)):
            for k in range(len(type_list)):
                path=year_list[i]+'/'+year_list[i]+"_"+month_list[j]+type_list[k]+'/'
                path_list.append(path)

#페이지 url을 입력하면 
def problem_crop(image_url,page_num):

    #이미지url을 open CV에 읽어들이기 
    imgfile = image_url
    image = cv2.imread(imgfile)


    #한 페이지를 반페이지로 나누기 
    left,right = imtrim(image,page_num)
    
    #각 반페이지에 대해 크롭 진행 

    contour(left)
    contour(right)








In [ ]:
#함수실행 :수능완성 
if __name__ == "__main__":
    path_list=[]
    make_path_SW()
    #make_path()
    path_prev='/gdrive/MyDrive/ProjectStudy/answers/'
    path_back='/Suwan_na/'
    

    for i in range(len(path_list)):
    #
        qnum=1
    #
        load_path=path_prev+'raw_data'+path_back+path_list[i]
        save_path=path_prev+'cropped'+path_back+path_list[i]
        os.chdir(load_path)
    
    #
        images=glob.glob('./*.png')

        #폴더내 이미지 순서대로 불러오기
        images.sort()
        #print(images)
       
    #
        qans_page_list=[]
        for i in range(len(images)):
        #
            filename=images[i]
        #
            page_num=filename[20:23]
            j=page_num
            
            print(filename)
            problem_crop(filename,j)